# Computing MO overlap matrix in ORCA and OpenMolcas


Computation of the MO time-overlaps is an important component in nonadiabatic dynamics. For computing the MO overlap integrals, usually a "double-molecule" approach is adopted which is done by requesting the quantum chemistry package to compute the AO overlap matrix for two nearby geometries in the MD trajectory. This procedure is used by different nonadiabatic dynamics codes such as SHARC, Newton-X, and pyUnixMD. 

Some other codes such as Libra and nano-qmflows can compute the overlap integrals using an analytical approach by explicitly computing the AO overlap matrix using Libint code which is an OpenMP parallelized code for computing different types of integrals in Gaussian-type orbital basis. Although both packages use Libint but the programming details are quite different. While nano-qmflows can perform the calculation only for non-periodic systems in DFT framework, Libra extends the calculations for periodic systems and in different approaches such as extended tight-binding.

Libra uses `molden` file format for computing the overlap matrices. In this notebook, I will show the capability of the Libra functions for computing the overlap using the `molden` files produced by ORCA and OpenMolcas. This interface will be generalized for time-overlaps and CI overlaps.

## 1. ORCA

I have generated a `molden` file format, `orca_molden.molden`, for a CH4 molecule using ORCA. Using the same procedure as was shown in 

In [1]:
import numpy as np
from liblibra_core import *
from libra_py import CP2K_methods, data_conv, molden_methods, orca_methods

/projects/academic/alexeyak/mohammad/software/miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/projects/academic/alexeyak/mohammad/software/miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/projects/academic/alexeyak/mohammad/software/miniconda3/envs/libra/lib/python3.6/import

### Creating the integration shell and generating the eigenvectors

In [2]:
shell_1, l_vals = molden_methods.molden_file_to_libint_shell('orca_molden.molden', True, True, np.array([[0,0,0],[0,0,0],[0,0,0]]), np.array([0,0,0]))
eig_vect_1, energies_1 = molden_methods.eigenvectors_molden('orca_molden.molden', nbasis(shell_1), l_vals)

### Computing the AO overlap matrix

In [3]:
AO_S = compute_overlaps(shell_1, shell_1, 12)
AO_S = data_conv.MATRIX2nparray(AO_S)

### Sorting the eigenvectors

The ordering of the angular momentum in ORCA should be changed based on the how Libra computes the integral. The ordering in ORCA is `pz, px, py, dz2, dxz, dyz, dx2y2, dxy` while the ordering in Libra/Libint is `px, py, pz, dyz, dx2y2, dxz, dxy, dz2`.

In [4]:
p_perm = [2,3,1]
d_perm = [3, 4, 2, 5, 1]

new_indices = orca_methods.resort_molog_eigenvectors(l_vals, list(p_perm), list(d_perm))
eigenvectors_1 = []
for j in range(len(eig_vect_1)):
    eigenvector_1 = eig_vect_1[j]
    eigenvector_1 = eigenvector_1[new_indices]
    eigenvectors_1.append(eigenvector_1)

eigenvectors_1 = np.array(eigenvectors_1)

[2, 3, 1] [3, 4, 2, 5, 1]


### Computing the MO overlap matrix

To validate the results of the overlap calculations, we print the diagonal elements of the MO overlap matrix and check they show normalized eigenvectors.

In [5]:
S = np.linalg.multi_dot([eigenvectors_1, AO_S, eigenvectors_1.T])
print('Diagonal elements of S matrix for ORCA!!!! ')
print(np.diag(S))

Diagonal elements of S matrix for ORCA!!!! 
[1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j
 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j
 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j
 1.+0.j 1.+0.j 1.+0.j 1.+0.j]


## 2. OpenMolcas

I use the same procedure as above but this time for a `molden` file format generated by OpenMolcas - I borrowed one of the `molden` files produced by Prof. Sebastian Mai in `/projects/academic/cyberwksp21/Students/smai/molcas_tests/MOLCAS.guessorb.molden`.

The rest of the procedure is as the same as in ORCA with the difference that the ordering of the eigenvectors in OpenMolcas is the same as in CP2K.

In [6]:
shell_1, l_vals = molden_methods.molden_file_to_libint_shell('MOLCAS.guessorb.molden', True)
eig_vect_1, energies_1 = molden_methods.eigenvectors_molden('MOLCAS.guessorb.molden', nbasis(shell_1), l_vals)

AO_S = compute_overlaps(shell_1, shell_1, 12)
AO_S = data_conv.MATRIX2nparray(AO_S)

new_indices = CP2K_methods.resort_molog_eigenvectors(l_vals)
eigenvectors_1 = []
for j in range(len(eig_vect_1)):
    eigenvector_1 = eig_vect_1[j]
    eigenvector_1 = eigenvector_1[new_indices]
    eigenvectors_1.append(eigenvector_1)
eigenvectors_1 = np.array(eigenvectors_1)

S = np.linalg.multi_dot([eigenvectors_1, AO_S, eigenvectors_1.T])

print('Diagonal elements of S matrix for OpenMolcas!!!! ')
print(np.diag(S))


Diagonal elements of S matrix for OpenMolcas!!!! 
[1.00000001+0.j 1.00000001+0.j 1.        +0.j 1.        +0.j
 0.99999999+0.j 0.99999999+0.j 1.00000001+0.j 1.00000001+0.j
 1.        +0.j 1.00000001+0.j 1.        +0.j 1.        +0.j
 1.00000001+0.j 0.99999998+0.j 1.00000001+0.j 1.        +0.j
 1.        +0.j 1.        +0.j 1.        +0.j 1.        +0.j
 1.        +0.j 0.99999999+0.j 1.        +0.j 1.00000001+0.j
 1.        +0.j 1.        +0.j 1.        +0.j 1.        +0.j
 1.00000001+0.j 1.        +0.j 1.        +0.j 1.        +0.j
 1.        +0.j 1.00000001+0.j 1.00000001+0.j 1.        +0.j
 0.99999999+0.j 1.        +0.j 1.00000001+0.j 1.00000001+0.j
 1.        +0.j 1.        +0.j 1.        +0.j 1.        +0.j
 1.        +0.j 1.        +0.j]
